In [ ]:
!pip install osmnx folium requests pandas openweather-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.4 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
  Attempting uninstall: geopandas
    Found existing installation: geopandas 1.0.1
    Uninstalling geopandas-1.0.1:
      Successfully uninstalled geopandas-1.0.1


In [ ]:
import osmnx as ox
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# Define your OpenWeatherMap API key
OPENWEATHERMAP_API_KEY = 'ef8bf71f0149b8b49ac4ec1f8ff3a105'

# Define function to fetch weather data from OpenWeatherMap
def get_weather(lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPENWEATHERMAP_API_KEY}"
    response = requests.get(url)
    data = response.json()

    if response.status_code == 200:
        weather_info = {
            'weather_type': data['weather'][0]['main'],
            'temperature': data['main']['temp'] - 273.15,  # Convert from Kelvin to Celsius
            'visibility': data.get('visibility', None),     # Visibility in meters
            'wind_speed': data['wind'].get('speed', None)   # Wind speed in m/s
        }
        return weather_info
    else:
        return None


In [ ]:
# Define a bounding box around Chandigarh (example coordinates)
chandigarh_bbox = (30.6942, 76.7821, 30.7737, 76.8484)  # (south, west, north, east)

# Define a central point in Chandigarh (to get initial network)
central_point = (30.7333, 76.7794)


In [ ]:
def get_chandigarh_network(bbox, distance=5000):
    # Create a graph covering the entire bounding box for Chandigarh
    G = ox.graph_from_bbox(bbox[2], bbox[0], bbox[3], bbox[1], network_type='drive')
    nodes, edges = ox.graph_to_gdfs(G)
    return nodes, edges

# Use the updated function to get nodes and edges within Chandigarh
nodes, edges = get_chandigarh_network(chandigarh_bbox)


<ipython-input-7-e921280640fe>:3: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  G = ox.graph_from_bbox(bbox[2], bbox[0], bbox[3], bbox[1], network_type='drive')
<ipython-input-7-e921280640fe>:3: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = ox.graph_from_bbox(bbox[2], bbox[0], bbox[3], bbox[1], network_type='drive')


In [ ]:
def collect_chandigarh_data(nodes, edges):
    data = []
    for _, row in edges.iterrows():
        segment = {
            'segment_id': row['osmid'],
            'length': row['length'],
            'road_type': row['highway'],
            'geometry': row['geometry'],
            'start_lat': row['geometry'].coords[0][1],
            'start_lon': row['geometry'].coords[0][0],
            'end_lat': row['geometry'].coords[-1][1],
            'end_lon': row['geometry'].coords[-1][0]
        }

        # Fetch weather for the midpoint of each segment
        mid_lat = (segment['start_lat'] + segment['end_lat']) / 2
        mid_lon = (segment['start_lon'] + segment['end_lon']) / 2
        weather = get_weather(mid_lat, mid_lon)

        segment.update({
            'temperature': weather['temperature'] if weather else None,
            'weather_type': weather['weather_type'] if weather else None,
            'visibility': weather['visibility'] if weather else None,
            'wind_speed': weather['wind_speed'] if weather else None,
            'date_time': datetime.now()
        })

        data.append(segment)

        # Sleep to avoid hitting API limits
        time.sleep(1)

    return pd.DataFrame(data)

# Run the function and save the data
chandigarh_data = collect_chandigarh_data(nodes, edges)
chandigarh_data.to_csv('chandigarh_route_data.csv', index=False)


In [ ]:
import random

def simulate_traffic(data):
    traffic_conditions = ['low', 'medium', 'high']
    data['traffic_condition'] = data['length'].apply(lambda x: random.choice(traffic_conditions))
    data['average_speed'] = data['traffic_condition'].apply(lambda x: random.uniform(20, 40) if x == 'low' else random.uniform(10, 20) if x == 'medium' else random.uniform(5, 10))
    data['travel_time'] = data['length'] / data['average_speed'] * 60  # Travel time in minutes
    return data

# Apply simulated traffic to dataset
chandigarh_data = simulate_traffic(chandigarh_data)
chandigarh_data.to_csv('chandigarh_route_data_with_traffic.csv', index=False)



In [ ]:
from google.colab import files

# Download the CSV file
files.download('chandigarh_route_data_with_traffic.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>